## Methpype usage example
The following notebook details a usage example for the python-based Illumina methylation array preprocessing software, methpype, using an example 450k Illumina experiment. To learn more about the specifications of Illumina's EPIC array, check out their [support page](https://support.illumina.com/array/array_kits/infinium-methylationepic-beadchip-kit/downloads.html).

### Module import
First we need to import the methpype module.

In [1]:
# add directory of function to path
import sys
sys.path.append("/Users/nrigby/GitHub/methpype/methpype/")

# import functions from methpype
import utils as utils
import data_extraction as datext
import preprocess as preprocess
import postprocess as postprocess

### Reading the samplesheet
We begin the methpype pipeline by reading in the Illumina samplesheet. This is most often provided in the data output if you receive your Illumina data from an outsourced laboratory. methpype requires that the columns "Sentrix_Position" and "Sentrix_ID" are both in the samplesheet. These are used to uniquely identify each sample.

To read in the samplesheet, we'll call the `read_sample_sheet` from `methpype.utils`. This function takes only one argument: the full path to the base directory in which your samplesheet and associated IDAT files are stored.

We're going to be using some example data downloaded from NCBI's GEOdatasets. For simplicity, let's use a subset of [GSE100850](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE100850).

In [2]:
# read in the sample sheet for the experiment
baseDir = "/Users/nrigby/GitHub/methpype/docs/example_data/GSE100850/"
sheet = utils.read_sample_sheet(baseDir)

After reading in the samplesheet, we can view the results. As you can see this is stored as a pandas dataframe. The columns "Sentrix_Position" and "Sentrix_ID" have been renamed to "Slide" and "Array". Also, a column called "Basename" has been added, which contains the base path for each of the samples. This is used later to read in the IDAT files for each sample.

Your own samplesheet may have more columns than our example here. You may choose to store addtional phenotypic data for each sample in your samplesheet that can be used later for analysis. For this example, however, we're working with a small samplesize with few features.

In [3]:
sheet

,GSM_ID,Sample_Name,Slide,Array,Basename
0,GSM2695082,NormalBreastTissueOld,200526210010,R01C01,/Users/nrigby/GitHub/methpype/docs/example_dat...
1,GSM2695083,BreastCancerOld,200526210010,R02C01,/Users/nrigby/GitHub/methpype/docs/example_dat...


### Extracting the raw data
Next, we'll extract the raw, binary data from each of the IDAT files and store it in a RawDataset object. We can do this using the function `extract_raw_data` from `methpype.data_extraction`. This function takes only one argument, the samplesheet dataframe we created in the previous step.

In [4]:
raw_dataset = datext.extract_raw_data(sheet)

Reading IDAT file 1 of 4: /Users/nrigby/GitHub/methpype/docs/example_data/GSE100850/GSM2695082_200526210010_R01C01_Grn.idat
Reading IDAT file 2 of 4: /Users/nrigby/GitHub/methpype/docs/example_data/GSE100850/GSM2695082_200526210010_R01C01_Red.idat
Reading IDAT file 3 of 4: /Users/nrigby/GitHub/methpype/docs/example_data/GSE100850/GSM2695083_200526210010_R02C01_Grn.idat
Reading IDAT file 4 of 4: /Users/nrigby/GitHub/methpype/docs/example_data/GSE100850/GSM2695083_200526210010_R02C01_Red.idat
Determining array type and reading manifest file.
Reading manifest file for IlluminaHumanMethylationEPIC array


As you can see, the `extract_raw_data` function not only reads in the green and red IDAT files for each sample, but it also reads in the Illumina manifest file for our array type. 

Let's take a closer look at the RawDataset class.

In [5]:
# confirm data type
type(raw_dataset)

data_extraction.RawDataset

The RawDataset class works much like a dictionary (or a scikit-learn Bunch if you're familiar with that class). If we look at the keys in our RawDataset instance, we can see the type of data that is stored as a result of our raw data extraction.

In [6]:
raw_dataset.keys()

dict_keys(['idat_objects', 'manifest', 'manifest_columns', 'array', 'samplesheet', 'data_type'])

#### RawDataset attributes
>- **`idat_objects:`** A list of all of the IDAT objects associated with the dataset. There will be two IDAT files for each sample in the samplesheet: one red and one green.
>- **`manifest:`** A two-dimensional numpy array containing the Illumina manifest information for our particular array. In this case the manifest pertains to an Illumina 450K array.
>- **`manifest_columns:`** A one-dimensional numpy array containing the column headers for the manifest.
>- **`array:`** A string specifying the Illumina array type. 450K, EPIC and Custom Life EGX EPIC+ arrays are currently supported.
>- **`sample_sheet:`** Our samplesheet dataframe that we read in earlier.
>- **`data_type:`** String specifying the dataset type. In the case of a RawDataset, the data_type will be "Raw". This is used internally by later functions to determine the dataset type. 

Each attribute can be accessed using dot notation. For example, we can get the array type of the dataset from `RawDataset.array`.

In [7]:
raw_dataset.array

'IlluminaHumanMethylationEPIC'

In [8]:
# look at the idat names and colors - each sample should have a red and a green
for idat in raw_dataset.idat_objects:
    print(idat.barcode,idat.array_name,idat.color)

200526210010 R01C01 green
200526210010 R01C01 red
200526210010 R02C01 green
200526210010 R02C01 red


There are several methods associated with the RawDataset class that are designed for internal use, but can be called on their own.

In [9]:
# you can get the list of probe addresses that are common for all samples in the experiment
# look at the first 10
raw_dataset.get_common_addresses()[0:10]

[16777216,
 35651586,
 16777220,
 81788932,
 56623110,
 58720264,
 35651594,
 58720268,
 89617430,
 58720272]

In [10]:
# view the green probe mean intensities for all samples
green_means, green_columns = raw_dataset.get_color_means(color='green')
green_means

array([[16777216,     1551,     1229],
       [35651586,     1707,     1480],
       [16777220,     2925,     3416],
       ...,
       [16777210,      143,      171],
       [58720252,      830,     1962],
       [56623102,     1336,      957]])

In [11]:
green_columns

array(['Address', '200526210010_R01C01', '200526210010_R02C01'],
      dtype='<U19')

In [12]:
# view the red probe mean intensities
red_means, red_columns = raw_dataset.get_color_means(color='red')
red_means

array([[16777216,      505,      294],
       [35651586,     1193,      594],
       [16777220,     1280,      874],
       ...,
       [16777210,     1117,      855],
       [58720252,      269,      458],
       [56623102,      456,      602]])

In [13]:
red_columns

array(['Address', '200526210010_R01C01', '200526210010_R02C01'],
      dtype='<U19')

In [14]:
raw_dataset.get_manifest_info(type='nLoci')

865933

In [15]:
raw_dataset.get_manifest_info(type='locusNames')[0:10]

['cg15153141',
 'cg24712639',
 'cg11101316',
 'cg00651363',
 'cg17329035',
 'cg26999725',
 'cg01485189',
 'cg06038599',
 'cg11551740',
 'cg07090691']

In [16]:
raw_dataset.manifest

array([['cg07881041', 'cg07881041', '85713262', ..., '5236016.0', 'R',
        'TypeII'],
       ['cg18478105', 'cg18478105', '46761277', ..., '61847650.0', 'R',
        'TypeI'],
       ['cg23229610', 'cg23229610', '21717843', ..., '6841125.0', 'R',
        'TypeII'],
       ...,
       ['73635489', 'NORM_T', 'Purple', ..., '', '', 'TypeControl'],
       ['73784382', 'NORM_C', 'Green', ..., '', '', 'TypeControl'],
       ['73794434', 'NORM_C', 'Green', ..., '', '', 'TypeControl']],
      dtype='<U50')

### Preprocessing the data
The raw red and green probe intensities need to be translated into methylation and unmethylation values. We can do this using the manifest information for our array. This step is completed as part of methpype's preprocessing functionality.

In this preprocessing step, we can also choose to implement a normalization method to control for techinal and/or samples effects in the array. If we want no normalization, we can choose to run `preprocess_raw` from `methpype.preprocess`.

In [17]:
preprocessed_dataset = preprocess.preprocess_raw(raw_dataset)

Preprocessing functions in methpype produce another object: a PreprocessDataset class object. This works much like a RawDataset object, but the data stored is now methylated and unmethylated values rather than raw, probe intensities.

In [18]:
preprocessed_dataset.keys()

dict_keys(['methylation_columns', 'methylated', 'unmethylated', 'manifest', 'manifest_columns', 'array', 'preprocessing_method', 'mapped', 'samplesheet', 'data_type'])

#### PreprocessDataset attributes
>- **`methylation_columns:`** A one-dimensional numpy array containing the column headers for the methylated and unmethylated arrays.
>- **`methylated:`** Two-dimensional numpy array containing the methylated values for each probe for each sample.
>- **`unmethylated:`** Two-dimensional numpy array containing the unmethylated values for each probe for each sample.
>- **`manifest:`** A two-dimensional numpy array containing the Illumina manifest information for our particular array. In this case the manifest pertains to an Illumina 450K array.
>- **`manifest_columns:`** A one-dimensional numpy array containing the column headers for the manifest.
>- **`array:`** A string specifying the Illumina array type. 450K, EPIC and Custom Life EGX EPIC+ arrays are currently supported.
>- **`preprossing_method:`** String specifying the preprocessing method used to create the PreprocessDataset class.
>- **`mapped:`** Boolean specifying whether the data has been mapped back to its genomic location. (More on this below.)
>- **`sample_sheet:`** Our samplesheet dataframe that we read in earlier.
>- **`data_type:`** String specifying the dataset type. In the case of a PreprocessDataset, the data_type will be "Preprocess". This is used internally by later functions to determine the dataset type. 

Each attribute can be accessed using dot notation. For example, we can get the array type of the dataset from `RawDataset.array`.

In [19]:
preprocessed_dataset.methylation_columns

array(['Name', '200526210010_R01C01', '200526210010_R02C01'], dtype='<U19')

In [20]:
preprocessed_dataset.methylated

array([['cg17720087', '1707', '1480'],
       ['cg12131279', '2925', '3416'],
       ['cg01498090', '1533', '1201'],
       ...,
       ['cg17901106', '468', '494'],
       ['cg24676740', '4034', '4295'],
       ['cg16369651', '4015', '3897']], dtype='<U21')

In [21]:
preprocessed_dataset.unmethylated

array([['cg17720087', '1193', '594'],
       ['cg12131279', '1280', '874'],
       ['cg01498090', '1268', '1579'],
       ...,
       ['cg21848700', '2021', '2173'],
       ['cg05555286', '3628', '3749'],
       ['cg00865290', '2430', '2448']], dtype='<U21')

In [22]:
preprocessed_dataset.mapped

False

As you can see, we have not yet mapped our probes back to their genomic information. We can do this using data in manifest by calling the `map_to_genome` function from `methpype.utils`.

In [23]:
preprocessed_dataset = utils.map_to_genome(preprocessed_dataset)
preprocessed_dataset.mapped

True

Now our probes are mapped to their genomic locations, and we can see this in the updated methylated and unmethylated dataframes. Additionally, the manifest columns have been updated to include the chromosome, chromosome location, genome build and strand.

In [24]:
preprocessed_dataset.methylated

array([['cg17720087', '1707', '1480', ..., 'R', '37.0', '21964652.0'],
       ['cg12131279', '2925', '3416', ..., 'R', '37.0', '61951439.0'],
       ['cg01498090', '1533', '1201', ..., 'F', '37.0', '12885115.0'],
       ...,
       ['cg17901106', '468', '494', ..., 'F', '37.0', '2162412.0'],
       ['cg24676740', '4034', '4295', ..., 'F', '37.0', '33236289.0'],
       ['cg16369651', '4015', '3897', ..., 'R', '37.0', '55410420.0']],
      dtype='<U21')

In [25]:
preprocessed_dataset.methylation_columns

array(['Name', '200526210010_R01C01', '200526210010_R02C01', 'Chromosome',
       'Strand', 'Genome_Build', 'MapInfo'], dtype='<U19')

In [26]:
import numpy as np
data_to_write = np.vstack([preprocessed_dataset.methylation_columns.astype(str),preprocessed_dataset.methylated.astype(str)])
np.savetxt("methylated.csv", data_to_write, delimiter=',', fmt='%s')
data_to_write = np.vstack([preprocessed_dataset.methylation_columns.astype(str),preprocessed_dataset.unmethylated.astype(str)])
np.savetxt("unmethylated.csv", data_to_write, delimiter=',', fmt='%s')

### Data postprocessing 
The final steps in the methpype pipeline allow you to post-process your methylated and unmethylated values to commonly used measures in the field. M and Beta values are commonly used measures for reporting methylation levels.

#### Beta value
\begin{equation*}
\beta = \frac{M}{M+ U+ 100}
\end{equation*}

#### M value
\begin{equation*}
Mval = log_2\left(\frac{M}{U}\right)
\end{equation*}

Where M and U denote the methylated and unmethylated signals, respectively. 

Let's calculate Beta values using `get_beta` from `methpype.postprocess`.


In [27]:
b_values,b_columns = postprocess.get_beta(preprocessed_dataset)
b_values

array([['cg00000029', '0.17630597014925373', '0.238898756660746'],
       ['cg00000103', '0.3870967741935484', '0.5451788963007883'],
       ['cg00000109', '0.6396028037383178', '0.7565104166666666'],
       ...,
       ['ch.X.97651759F', '0.11287048315062931', '0.10126582278481013'],
       ['ch.X.97737721F', '0.11634887005649718', '0.12610812835559995'],
       ['ch.X.98007042R', '0.12900926585887385', '0.14935064935064934']],
      dtype='<U32')

In [28]:
b_columns

array(['Name', '200526210010_R01C01', '200526210010_R02C01'], dtype='<U19')